**LAB 1**

**SPAM CLASSIFICATION**

**PART 1:**
SPAM Filtering using Naive Bayes Classifier.


Importing all the relevant libraries.

In [ ]:
import string 
import os
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import json
from math import exp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The code below creates the dictionary of words by iterating through all the emails in the training set.

In [ ]:
def create_word_dictionary(train_dir):
  d = dict() 
  no_of_mails = 0
  emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)] 
  for mail in emails:
      no_of_mails = no_of_mails + 1; 
      text = open(mail, "r")  
      for line in text: 
          line = line.strip() 
          line = line.lower() 
          line = line.translate(line.maketrans("", "", string.punctuation)) 
          words = line.split(" ") 
          for word in words:
              if (len(word) > 2) or (word !=''): 
                  d[word] = 0
  d.pop('',None)
  return d, no_of_mails  

The code below is used to count the number of emails we are taking into consideration for training and test cases.

In [ ]:
def count_emails(dir):
  count = 0
  emails = [os.path.join(train_dir,f) for f in os.listdir(dir)]
  for mail in emails:
    count = count+1
  return count 

The function create_feature_vector_TF creates the term frequency feature vector by iteratng over all the emails and then counting how many times a word occurs in the mail and then creating an array of these term frequencies.

In [ ]:
def create_feature_vector_TF(train_dir, no_of_mails,d): 
  matrix = np.empty(shape=(no_of_mails,2),dtype='object')
  i=0
  emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]
  for mail in emails:
      text = open(mail, "r") 
      de = dict(d)
      dictlist =[]
      for line in text: 
          line = line.strip() 
          line = line.lower() 
          line = line.translate(line.maketrans("", "", string.punctuation)) 
          words = line.split(" ") 
          for word in words: 
              if (word in de) and (word != ''):
                  de[word] = de[word]+1
              else:
                  if word !='':
                      de[word] = 0
      for key in de.keys():
          temp = de[key]
          dictlist.append(temp)        
      if 'spm' in mail:
          matrix[i][0] = dictlist
          matrix[i][1] = 1
      else:
          matrix[i][0] = dictlist
          matrix[i][1] = 0
      i=i+1
  X_train = np.empty(shape =(np.shape(matrix)[0],len(matrix[0][0])), dtype = int)
  Y_train = np.empty(shape = (np.shape(matrix)[0],1))
  for l in range(0, np.shape(matrix)[0]):
    for k in range(0, len(matrix[0][0])):
        X_train[l][k] = (matrix[l][0])[k] 
    Y_train[l] = matrix[l][1]
  return X_train, Y_train


The function create_feature_vector_BF creates the feature vector by iteratng over all the emails and then checking if a word is present in the email or not and then creating an array of this information.

In [ ]:
def create_feature_vector_BF(train_dir, no_of_mails,d): 
  matrix = np.empty(shape=(no_of_mails,2),dtype='object')
  i=0
  emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]
  for mail in emails:
      text = open(mail, "r") 
      de = dict(d)
      dictlist =[]
      for line in text: 
          line = line.strip() 
          line = line.lower() 
          line = line.translate(line.maketrans("", "", string.punctuation)) 
          words = line.split(" ") 
          for word in words: 
              if (word in de) and (word != ''):
                  de[word] = 1
              else:
                  if word !='':
                      de[word] = 0
      for key in de.keys():
          temp = de[key]
          dictlist.append(temp)        
      if 'spm' in mail:
          matrix[i][0] = dictlist
          matrix[i][1] = 1
      else:
          matrix[i][0] = dictlist
          matrix[i][1] = 0
      i=i+1
  X_train = np.empty(shape =(np.shape(matrix)[0],len(matrix[0][0])), dtype = int)
  Y_train = np.empty(shape = (np.shape(matrix)[0],1))
  for l in range(0, np.shape(matrix)[0]):
    for k in range(0, len(matrix[0][0])):
        X_train[l][k] = (matrix[l][0])[k] 
    Y_train[l] = matrix[l][1]
  return X_train, Y_train

The function below calculates the information gain for the features in the training set.

In [ ]:
def calc_information_gain(X_train, Y_train):
  ig = mutual_info_classif(X_train,Y_train.ravel())
  id_sort = np.argsort(ig, axis=-1)
  return ig, id_sort
# print(ig)
# print(np.shape(X_train))
# print(np.shape(ig))



Based on the Information Gain value, the code below selects N number of features with the highest IG value.

In [ ]:
def select_features(ig, id_sort, N, X_train):
  FS_X_train = np.empty(shape = (np.shape(X_train)[0], N), dtype = int)

  for p in range(0, np.shape(X_train)[0]):
      z=0
      for s in id_sort[-N:]:
          FS_X_train[p][z] = X_train[p][s]
          z = z+1
  return FS_X_train

The code that is commented out below is caling the create_dictionary function and then writing the dictionary to a json file so that we don't have to create a dictionary everytime we run the code.

In [ ]:
train_dir = '/content/drive/My Drive/train'
test_dir = '/content/drive/My Drive/test'

# print("Creating dictionary of words...")
# d, no_of_train_mails = create_word_dictionary(train_dir)
# print("Dictionary creation complete.")
# with open('/content/drive/My Drive/my_dict.json', 'w') as f:
#     json.dump(d, f)

The code below loads the json file(which was created in the previous step) as the dictionary.

In [ ]:
with open('/content/drive/My Drive/my_dict.json') as f:
    d = json.load(f)

no_of_train_mails = count_emails(train_dir)

The code below creates the training set and test set for both binary features and term frequency features.
This codes takes a lot of time(around 15-20 mins) to execute because the function that calculates the information gain is very intensive.

In [ ]:
#Binary Features
print("Now, creating Binary training feature vector...")
X_train_BF, Y_train_BF = create_feature_vector_BF(train_dir, no_of_train_mails, d)
print("Training feature vector creation complete.")
no_of_test_mails = count_emails(test_dir)
print("Now, creating test feature vector...")
X_test_BF, Y_test_BF = create_feature_vector_BF(test_dir, no_of_test_mails, d)
print("Test feature vector creation complete.")

print("Calculating IG values...")
ig_BF,id_sort_BF = calc_information_gain(X_train_BF, Y_train_BF)
print("IG calculation complete.")

#Term Frequency Features
print("Now, creating Term Frequency training feature vector...")
X_train_TF, Y_train_TF = create_feature_vector_TF(train_dir, no_of_train_mails, d)
print("Training feature vector creation complete.")
no_of_test_mails = count_emails(test_dir)
print("Now, creating test feature vector...")
X_test_TF, Y_test_TF = create_feature_vector_BF(test_dir, no_of_test_mails, d)
print("Test feature vector creation complete.")

print("Calculating TF IG values...")
ig_TF,id_sort_TF = calc_information_gain(X_train_TF, Y_train_TF)
print("IG calculation complete.")


Now, creating Binary training feature vector...
Training feature vector creation complete.
Now, creating test feature vector...
Test feature vector creation complete.
Calculating IG values...
IG calculation complete.
Now, creating Term Frequency training feature vector...
Training feature vector creation complete.
Now, creating test feature vector...
Test feature vector creation complete.
Calculating TF IG values...
IG calculation complete.


We are fitting our Naive Bayes Model to the data that we created above. The list N is the list of features to be used for training and testing the ML algorithm. pr_list is a list of precision and recall values for all the different classifiers and different feature sets.

In [ ]:
N=[10, 100, 1000]
clf1 = BernoulliNB()
clf2 = MultinomialNB()
pr_list = []

for n in N:
  pr = []
  FS_X_train = select_features(ig_BF, id_sort_BF, n, X_train_BF)
  FS_X_test = select_features(ig_BF, id_sort_BF, n, X_test_BF)

  clf1.fit(FS_X_train, Y_train_BF.ravel())
  Yhat1 = clf1.predict(FS_X_test)
  print("Accuracy: ", clf1.score(FS_X_test, Y_test_BF.ravel()))
  precision, recall, _, _ = precision_recall_fscore_support(Y_test_BF, Yhat1)
  pr.append(precision[1])
  pr.append(recall[1])
  pr_list.append(pr)

for n in N:
  pr = []
  FS_X_train = select_features(ig_BF, id_sort_BF, n, X_train_BF)
  FS_X_test = select_features(ig_BF, id_sort_BF, n, X_test_BF)

  clf2.fit(FS_X_train, Y_train_BF.ravel())
  Yhat2 = clf2.predict(FS_X_test)
  print("Accuracy: ", clf2.score(FS_X_test, Y_test_BF.ravel()))
  precision, recall, _, _ = precision_recall_fscore_support(Y_test_BF, Yhat2)
  pr.append(precision[1])
  pr.append(recall[1])
  pr_list.append(pr)

for n in N:
  pr = []
  FS_X_train_TF = select_features(ig_TF, id_sort_TF, n, X_train_TF)
  FS_X_test_TF = select_features(ig_TF, id_sort_TF, n, X_test_TF)

  clf2.fit(FS_X_train_TF, Y_train_TF.ravel())
  Yhat3 = clf2.predict(FS_X_test_TF)
  print("Accuracy:", clf2.score(FS_X_test_TF, Y_test_TF.ravel()))
  precision, recall, _, _ = precision_recall_fscore_support(Y_test_TF, Yhat3)
  pr.append(precision[1])
  pr.append(recall[1])
  pr_list.append(pr)

Accuracy:  0.9621993127147767
Accuracy:  0.9553264604810997
Accuracy:  0.9621993127147767
Accuracy:  0.9518900343642611
Accuracy:  0.9828178694158075
Accuracy:  0.993127147766323
Accuracy: 0.9553264604810997
Accuracy: 0.9828178694158075
Accuracy: 0.9896907216494846


In [ ]:
for i in range(0,len(pr_list)):
  print(pr_list[i])


[0.8958333333333334, 0.8775510204081632]
[1.0, 0.7346938775510204]
[1.0, 0.7755102040816326]
[0.926829268292683, 0.7755102040816326]
[0.9782608695652174, 0.9183673469387755]
[1.0, 0.9591836734693877]
[0.8333333333333334, 0.9183673469387755]
[0.9583333333333334, 0.9387755102040817]
[1.0, 0.9387755102040817]


The code below prints the list of top-10 words with the highest IG value.

In [ ]:
top10_BF=[]
top10_TF=[]
words=list(d.keys())
N=10
for s in id_sort_BF[-N:]:
        top10_BF.append(words[s])
print(top10_BF)

for s in id_sort_TF[-N:]:
        top10_TF.append(words[s])
print(top10_TF)

['save', 'english', 'money', 'our', 'click', 'linguistic', 'university', 'free', 'remove', 'language']
['market', 'click', 'business', 'our', 'money', 'university', 'linguistic', 'free', 'remove', 'language']


**Part 2.**

**SVM Classifier**

The code below is fitting our training set to an SVM classifier. To find the optimal parameters of the SVM Classifier, I am using 10-fold cross validation. The features are **Binary Features** because they provide a higher accuracy score than term frequency features. The Features are selected based on the highest **IG value**. So, the first 100 features with highest IG values are selected. This is because **IG value** provides a good estimate of how much information the presence and absence of each words contributes to the email being a spam or ham. I chose the 100 terms for the feature set because that provides a balance between accuracy and computation time.

The GridSearchCV function is a very useful function provided by the sci-kit learn library that fits multiple models on a cross validation set. Multiple model refers to the same classifier with different parameters. The parameter array is a list of parameters that the GriSearchCV function uses to create the multiple models and then returns the parameter that give the best performance. 

The accuracy of the classifier with both TF and BF features can be seen below.

We can see that the SVM model performs best when we use the radial basis function kernel and the regularization parameter is set to 100.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

FS_X_train_BF = select_features(ig_BF, id_sort_BF, 100, X_train_BF)
FS_X_test_BF = select_features(ig_BF, id_sort_BF, 100, X_test_BF)

FS_X_train_TF = select_features(ig_TF, id_sort_TF, 100, X_train_TF)
FS_X_test_TF = select_features(ig_TF, id_sort_TF, 100, X_test_TF)


parameters = {'kernel':('linear', 'rbf', 'poly'), 'C':[1,5,10], 'degree':[1,2,3]}
svc = SVC()
clf1 = GridSearchCV(svc, parameters, cv=10)
clf1.fit(FS_X_train_TF, Y_train_TF.ravel())
Yhat_svm_TF = clf1.predict(FS_X_test_TF)
print(clf1.best_estimator_)
print("Accuracy: ", clf1.score(FS_X_test_TF, Y_test_TF.ravel()))
precision_TF, recall_TF, _, _ = precision_recall_fscore_support(Y_test_TF, Yhat_svm_TF)
print("Precision: ", precision_TF)
print("Recall: ", recall_TF)

clf2 = GridSearchCV(svc, parameters, cv=10)
clf2.fit(FS_X_train_BF, Y_train_BF.ravel())
Yhat_svm_BF = clf2.predict(FS_X_test_BF)
print(clf2.best_estimator_)
print("Accuracy: ", clf2.score(FS_X_test_BF, Y_test_BF.ravel()))
precision_BF, recall_BF, _, _ = precision_recall_fscore_support(Y_test_BF, Yhat_svm_BF)
print("Precision: ", precision_BF)
print("Recall: ", recall_BF)


SVC(C=5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=1, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Accuracy:  0.979381443298969
Precision:  [0.98360656 0.95744681]
Recall:  [0.99173554 0.91836735]
SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=1, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Accuracy:  0.9828178694158075
Precision:  [0.98367347 0.97826087]
Recall:  [0.99586777 0.91836735]


**Adversarial Classification:**
**Attack and Defense** 

The code below take the top 10 fetures with highest IG value and uses it to train a Multinomial NB classifier.

In [ ]:
clf2 = MultinomialNB()
n=10
FS_X_train = select_features(ig_BF, id_sort_BF, n, X_train_BF)
FS_X_test = select_features(ig_BF, id_sort_BF, n, X_test_BF)

clf2.fit(FS_X_train, Y_train_BF.ravel())
Yhat2 = clf2.predict(FS_X_test)
print("Accuracy before modification: ", accuracy_score(Y_test_BF, Yhat2))


Accuracy before modification:  0.9518900343642611


In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [ ]:
TP,FP,TN,FN = perf_measure(Y_test_BF, Yhat2)

print ("False Negative Rate before attacker's modification: ", FN/(FN+TP))

False Negative Rate before attacker's modification:  0.22448979591836735


The code below calculates the minimum cost that the attacker/adversary has to pay so that the classifier classifies all the spam email as legit emails.

In [ ]:
def mcc(abc, cost, p):
  abcd = abc
  abc = abcd + sorted_Loc[0][p]
  cost = cost+1;
  if abc>=0:
    cost = mcc(abc, cost, p+1)
  return cost

We are only considering 10 words in our machine learning model. This, according to me, is a limitation. This is because the words with highest IG value are the words that consist of info that distinguishes a spam email from a ham email. It could be a spam word or a ham word. The log odds of a ham word in the top 10 words would be negative and that of a spam word would be positive. According to the log odds calculation, only three words give information about the email being a ham. Since we only have 10 words and we can, at max, only add 3 of them to make a email get classified as spam instead of ham. If the email has a very high Loc value then adding the three words would not make the email classify it as legit. Adding other words will not have any effect because the classifier is only using these 10 words to determine if the email is spam or ham.

In [ ]:
p_xi_spam =np.zeros((1,10))
p_xi_legit = np.zeros((1,10))
for k in range(0,len(clf2.feature_log_prob_[0])):
  p_xi_legit[0][k] = exp(clf2.feature_log_prob_[0][k])

# print(p_xi_spam)
for k in range(0,len(clf2.feature_log_prob_[0])):
  p_xi_spam[0][k] = exp(clf2.feature_log_prob_[1][k])

Loc = np.log((p_xi_spam/p_xi_legit))

# print(p_xi_legit)
Lox = np.zeros(np.shape(FS_X_test))
i=0
for l in range(0, np.shape(FS_X_test)[0]):
  Lox[l] = (np.multiply(np.log((p_xi_spam/p_xi_legit)), FS_X_test[l]))


delta = np.zeros(np.shape(FS_X_test))
for i in range(0, np.shape(FS_X_test)[0]):
  for j in range(0, 10):
    delta[i][j] = max((Lox[i][j] - (1-Lox[i][j])),0)

# print(delta)

sorted_Loc = np.sort(Loc)
idx = np.argsort(Loc)
lis =[sorted_Loc, idx]


cost=[]
change_list = []
for t in range(0, np.shape(Yhat2)[0]):
    if Yhat2[t]==1:
      if np.sum(Lox[t])<13.5:
        change_list.append(t)
        cost_each = mcc(np.sum(Lox[t]),0, 0)
        cost.append(cost_each)

print("Cost for each spam email to make it ham is: ",cost)
print("Average cost of attacker is: ", np.average(cost))

for w in range(0, np.shape(Yhat2)[0]):
  if Yhat2[w] ==1:
    for z in cost:
      for y in range(0,z):
        FS_X_test[w][idx[0][y]] = 1;

Yhat_ad = clf2.predict(FS_X_test)
print("Accuracy after adversary's attack: ", accuracy_score(Y_test_BF, Yhat_ad))

TP,FP,TN,FN = perf_measure(Y_test_BF, Yhat_ad)

print ("False Negative Rate after attacker's modification: ", FN/(FN+TP))

Cost for each spam email to make it ham is:  [2, 3, 1, 2, 3, 3, 1, 1, 1, 2, 1, 1, 4, 3, 2, 2, 1, 1, 2, 3, 2, 2, 3, 2, 4, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 4, 1, 2, 3]
Average cost of attacker is:  1.9268292682926829
Accuracy after adversary's attack:  0.8316151202749141
False Negative Rate after attacker's modification:  1.0


Below is the code that the defender modifies to protect against the adversary's modification. However, the accuracy is very low because we are only considering 10 words. Lowering the threshold will also have an effect on legit emails being classified as spam. This is why we get low accuracy.

In [ ]:
New_Sum_Lox = np.zeros((np.shape(FS_X_test)[0],1))
for l in range(0, np.shape(FS_X_test)[0]):
  New_Sum_Lox[l] = np.sum(np.multiply((np.log(p_xi_spam/p_xi_legit)), FS_X_test[l]))

change_mail = np.zeros((len(change_list),1))
k=0
for i in change_list:
  change_mail[k] = New_Sum_Lox[i]
  k+=1

New_threshold = np.min(change_mail)

yhat_thresh_changed = np.zeros(np.shape(Yhat2))

for l in range(0, np.shape(X_test_BF)[0]):
  if New_Sum_Lox[l]>New_threshold:
    yhat_thresh_changed[l] = 1
  else:
    yhat_thresh_changed[l] = 0

TP,FP,TN,FN = perf_measure(Y_test_BF, yhat_thresh_changed)

print("False Positive Rate of updated classifier: ", FP/(FP+TP))
print("False Negative Rate of updated classifier: ", FN/(FN+TP))



False Positive Rate of updated classifier:  0.8203125
False Negative Rate of updated classifier:  0.061224489795918366
